In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import warnings
import os
import geopandas as gpd
import shapely
import rasterio 

from rasterio.features import geometry_mask
from PIL import Image
from scipy.ndimage import binary_erosion
from scipy.ndimage import binary_dilation

In [3]:
# Choose input parameters: 
name = "Sophia" # Kathrin, Philipp
site = "kolguev" # kolguev, taymyr, herschel, tuktoyaktuk, Peel
mode_dilation_erosion = 'none' # erosion, dilation, none: none calulates the actual volume
pixelresolution = 10 # 10 m for difference DEM generated with method 3, 6 m for method 1
# structuring element for dilation / erosion
structuring_element = np.array([[0, 1, 0],
                                      [1, 1, 1],
                                      [0, 1, 0]], dtype=bool)

# Set path to data
local_folder = os.getcwd()
file_folder = r'C:\Users\sophi\Documents\thesis\code\comparing labels\labeled_data'  # path to polygons extracted from QGIS labelling process

In [4]:
# Read QGIS data--------------------------------------- 
if site == "Peel":
    network_folder = r'Z:\Kathrin\dem_processing\dem_differencing\method_1'  # UNC path to dem tiles
    tile_path = os.path.join(local_folder, network_folder) # Construct the full path to the network folder from the local directory
    # Extract tile names and polygon
    tile_name = [item for item in os.listdir(os.path.join(tile_path))]
    year_start_ = [2010, 2011]
    year_end = [2016, 2021]
elif site == "herschel":
    network_folder = r'C:\Users\sophi\Documents\thesis\code\comparing labels\labeled_data\herschel'  # UNC path to dem tiles
    tile_path = os.path.join(local_folder, network_folder)
    tile_name = ['tile_18_42_7_6']
    year_start_ = [2010]
    year_end = [2016, 2018]
elif site == "kolguev":
    network_folder = r'C:\Users\sophi\Documents\thesis\code\comparing labels\labeled_data\kolguev'  # UNC path to dem tiles
    tile_path = os.path.join(local_folder, network_folder)
    tile_name = ['tile_63_42_7_4']
    year_start_ = [2010]
    year_end = [2017]

elif site == "taymyr":
    network_folder = r'C:\Users\sophi\Documents\thesis\code\comparing labels\labeled_data\taymyr_south_1'  # UNC path to dem tiles
    tile_path = os.path.join(local_folder, network_folder)
    tile_name = ['tile_54_53_2_6', 'tile_54_53_4_9']
    year_start = [2010]
    year_end = [2016, 2020]

elif site == "tuktoyaktuk":
    network_folder = r'C:\Users\sophi\Documents\thesis\code\comparing labels\labeled_data\tuktoyaktuk'  # UNC path to dem tiles
    tile_path = os.path.join(local_folder, network_folder)
    tile_name = ['tile_18_40_2_6', 'tile_18_40_4_8']
    year_start_ = [2010]
    year_end = [2016, 2021]


polygons = gpd.read_file(os.path.join(local_folder, file_folder, f"{site}_{name}.geojson")) # polygons extracted from QGIS labelling process

# Calculate shape attributes---------------------------------------
# Calculate the area and perimeter for each geometry
polygons['area'] = polygons['geometry'].apply(lambda x: x.area)
polygons['perimeter'] = polygons['geometry'].apply(lambda x: x.length)

# Calculate circularity using precomputed area and perimeter
polygons['circularity'] = (4 * np.pi * polygons['area']) / (polygons['perimeter'] ** 2)

# Calculate solidity using precomputed area and convex hull area
polygons['solidity'] = polygons['area'] / polygons['geometry'].apply(lambda x: x.convex_hull.area)

polygons["diameter_max"] = 2 * shapely.minimum_bounding_radius(polygons["geometry"])
polygons["n_vertices"] = shapely.get_num_coordinates(polygons["geometry"])

c:\Users\sophi\anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\Users\sophi\anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\Users\sophi\anaconda3\envs\geoprocessing\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, con

In [6]:
# Fix wrong columnname
if site == 'tuktoyaktuk':
    polygons.rename(columns={'tile_range': 'tile_name'}, inplace=True)

# change start year and end to int
polygons['year_start'] = polygons['year_start'].astype(int)
polygons['year_end'] = polygons['year_end'].astype(int)
print("Changed year from str to int")
year_start = list(polygons['year_start'].unique())
year_end = list(polygons['year_end'].unique())

Changed year from str to int


In [8]:
# Check number of unique RTSs
unique = polygons["id"].drop_duplicates(keep=False)
boolean_unique = polygons["id"].isin(unique)
non_unique = polygons["id"][~boolean_unique]
print(f"{len(non_unique)} RTS appear more than once in df (Because it overlaps multiple tiles). Each id is a unique RTS, we have {len(unique)} unique RTS")


0 RTS appear more than once in df (Because it overlaps multiple tiles). Each id is a unique RTS, we have 15 unique RTS


In [8]:
# Calculate RTS attributes based on (normalized) difference DEM
# Dictionary that keeps track of RTS that appear in multiple images: key = RTS index in polygon df
# dictionary = key accesses list of dictionary where each dictionary is metric from an image
RTS_multiple = {}
RTS_checked = []
stop_ = False

# iterate through tiles------------------------------------------------------------------------
for tile in tile_name:

    # Extract sub_df according to tilename
    sub_polygons_image = polygons[polygons["tile_name"] ==  tile]
    #print(len(sub_polygons_image))
    if len(sub_polygons_image)>0:
        # iterate through all combination of years------------------------------------------------------------------------
        # Makes sure that all tiles are read
        for start in year_start:
            for end in year_end:
                sub_polygons = sub_polygons_image[(sub_polygons_image["year_start"] == start) & (sub_polygons_image["year_end"] == end)]
                #print('y_start', start, 'year_end', end, 'tile', tile)
                # Only read image if there is at least one RTS in image to avoid unnecessary tasks
                
                if len(sub_polygons)>0:
                    # Read corresponding norm dem image---------- 
                    # Extract dem name corresponding to tile, start, end dem
                    if site == 'Peel' or site == 'taymyr' or site == 'tuktoyaktuk':
                        norm_dem_name = [item for item in os.listdir(os.path.join(tile_path, tile))
                                                if item.startswith("norm_diff_dem") and item.endswith(".tif") and str(start) in item and str(end) in item]
                        diff_dem_name = [item for item in os.listdir(os.path.join(tile_path, tile))
                                                if item.startswith("diff_dem") and item.endswith(".tif") and str(start) in item and str(end) in item]
                    else:
                        norm_dem_name = [item for item in os.listdir(tile_path)
                                                if item.startswith("norm_diff_dem") and item.endswith(".tif") and str(start) in item and str(end) in item]
                        diff_dem_name = [item for item in os.listdir(os.path.join(tile_path))
                                                if item.startswith("diff_dem") and item.endswith(".tif") and str(start) in item and str(end) in item]
                    if len(norm_dem_name)>1:
                        warnings.warn("There are more than one image that correspond to the wanted tile. The first", UserWarning)

                    if len(norm_dem_name)<1:
                        warnings.warn(f"RTS is in a non existing {tile, start, end}. Skip RTS calculations", UserWarning)
                        continue
                        
                    #print(norm_dem_name)
                    # Read image as georeferenced tif
                    # Read norm dem
                    if site == 'Peel'or site == 'taymyr' or site == 'tuktoyaktuk': # Split into subfolders
                        norm_dem_path = os.path.join(tile_path, tile, norm_dem_name[0]) 
                        diff_dem_path = os.path.join(tile_path, tile, diff_dem_name[0]) 
                        #print(norm_dem_path)
                    else: # All in one folder
                        norm_dem_path = os.path.join(tile_path, norm_dem_name[0])
                        diff_dem_path = os.path.join(tile_path, diff_dem_name[0]) 
                    src = rasterio.open(norm_dem_path)
                    norm_dem = src.read(1)
                    transform_norm = src.transform #transformation from pixel coordinates of source to the coordinate system of the input shapes

                    src_diff = rasterio.open(diff_dem_path)
                    diff_dem = src_diff.read(1)
                    transform_diff = src_diff.transform #transformation from pixel coordinates of source to the coordinate system of the input shapes

                    # Iterate through all RTS that are within the image-----------------------------------------------------------------------
                    for index, row in sub_polygons.iterrows():    
                        # Intersect polygons with norm dem image---------- 
                        geom = row['geometry']
                        mask = geometry_mask([geom], out_shape=norm_dem.shape, transform = transform_norm, invert=True) # invert to make sure that mask is 1 and background = 0
                        norm_dem_values = norm_dem[mask]

                        if mode_dilation_erosion =='dilation':
                            mask_diff = geometry_mask([geom], out_shape=diff_dem.shape, transform = transform_diff, invert=True) # invert to make sure that mask is 1 and background = 0
                            mask_diff = binary_dilation(mask_diff, structure=structuring_element)
                            diff_dem_values = diff_dem[mask_diff]   
                            area = np.sum(mask_diff) *(pixelresolution*pixelresolution)
                        elif  mode_dilation_erosion =='erosion':
                            mask_diff = geometry_mask([geom], out_shape=diff_dem.shape, transform = transform_diff, invert=True) # invert to make sure that mask is 1 and background = 0
                            mask_diff = binary_erosion(mask_diff, structure=structuring_element)
                            diff_dem_values = diff_dem[mask_diff]  
                            area = np.sum(mask_diff) *(pixelresolution*pixelresolution)
                        else:
                            # Intersect RTS polygon with difference DEM: original shape and buffered shape to take lower volume into account
                            mask_diff = geometry_mask([geom], out_shape=diff_dem.shape, transform = transform_diff, invert=True) # invert to make sure that mask is 1 and background = 0
                            diff_dem_values = diff_dem[mask_diff]     
                            area =row['area']                 
                        
                        # Calculate metrics
                        norm_mean = np.nanmean(norm_dem_values)
                        norm_std = np.nanstd(norm_dem_values)
                        norm_var = np.nanvar(norm_dem_values)
                        norm_median = np.nanmedian(norm_dem_values)
                        norm_coeff_var = norm_std / norm_var
                        norm_n_pixel = norm_dem_values.size
                        if not norm_n_pixel >0: # ensure that RTS is > 0
                            print('stop')
                            stop_ = True
                            break

                        # Calculate volume
                        height_mean = np.nanmean(diff_dem_values)
                        volume = area * height_mean

                        # Check if RTS has already been checked. If yes -> RTS is on multple tiles: add values to dictionary: key = RTS_id, values = intensity metric
                        # ---> If RTS appears in multiple picitures, we add values into a dictionary   
                        RTS_id = polygons.at[index, "id"]

                        if RTS_id in RTS_checked:
                            metric_dict = {
                                "norm_mean": norm_mean,
                                "norm_std": norm_std, 
                                "norm_var": norm_var,
                                "norm_median":  norm_median,
                                "norm_coeff_var": norm_coeff_var,
                                "norm_n_pixel": norm_n_pixel,
                                "volume": volume                        
                            }

                            # Check if the RTS_id exists in the dictionary (is already a  key): RTS is in > 2 pictures. Add 3rd image as another dictionary
                            if RTS_id in RTS_multiple:
                                RTS_multiple[RTS_id].append(metric_dict)
                            else:
                                RTS_multiple[RTS_id] = [metric_dict]
                            
                        # RTS has not been checked before: add values into polygon df
                        else:
                            polygons.at[index, "norm_mean"] =norm_mean
                            polygons.at[index, "norm_std"] = norm_std
                            polygons.at[index, "norm_var"] = norm_var
                            polygons.at[index, "norm_median"] = norm_median
                            polygons.at[index, "norm_coeff_var"] = norm_coeff_var
                            polygons.at[index, "norm_n_pixel"] = norm_n_pixel
                            polygons.at[index, "volume"] = volume

                        # Mark RTS as checked
                        RTS_checked.append(polygons.at[index, "id"])
                if stop_:
                    break
            if stop_:
                break
    if stop_:
        break

In [9]:
# Some RTS appear in multiple tiles -> For intensity metrics we calculate the weighted mean for each instance
# First, all values of a RTS have to be combined from RTS_multiple dictionary and polygons df
# RTS_multiple dictionary contains metric values of RTS instances that appear for the second, third... times. First appearance of RTS is saved in polygons df

polygons_aggregated = polygons.copy(deep=True)
# Extract RTS id that appear in multiple tiles
id_multiple = list(RTS_multiple.keys())

# Iterate through RTS that appear in multiple tiles
for id_RTS_m in id_multiple:
    # Create an empty aggregation df and fill first instance with first appearance of RTS from polygons df
    # Extract intensity metrics (polygons_aggregated.columns[12:]) from corresponding RTS
    same_RTS = polygons_aggregated[polygons_aggregated["id"]== id_RTS_m][polygons_aggregated.columns[12:]] 
    n_multiple = len(RTS_multiple[id_RTS_m]) + 1 # Get number of same instance in different tiles. +1 because first instance of RTS is saved in polygons df
    aggregation = pd.DataFrame(None,  index=range(n_multiple), columns=polygons_aggregated.columns[12:])
    aggregation.iloc[0] = same_RTS.iloc[0]

    # Fill other appearance of RTS from RTS_multiple dictionary
    for i in range (1, n_multiple): # First RTS is already assigned in df. Dictionary only contains value from second RTS instance on
        RTS_i_dict = RTS_multiple[id_RTS_m][i-1] 
        # Save all values from dictionary to df
        for col, value in RTS_i_dict.items(): 
            aggregation.at[i, col] = value
        # Calculate weighted mean and insert back into polygons_aggregated df
        aggregated = np.average(aggregation, axis = 0, weights = aggregation["norm_n_pixel"])
        polygons_aggregated.loc[polygons_aggregated['id'] == id_RTS_m, list(polygons_aggregated.columns[12:])] =aggregated
# Remove rows where id is not unique
polygons_aggregated = polygons_aggregated[~polygons_aggregated['id'].duplicated(keep='first')]

In [10]:
unique = polygons_aggregated["id"].drop_duplicates(keep=False)
boolean_unique = polygons_aggregated["id"].isin(unique)
non_unique = polygons_aggregated["id"][~boolean_unique]
print(f"{len(non_unique)} RTS appear more than once in df (Because it overlaps multiple tiles). Each id is a unique RTS, we have {len(unique)} unique RTS")


0 RTS appear more than once in df (Because it overlaps multiple tiles). Each id is a unique RTS, we have 99 unique RTS


In [11]:
# Remove rows where intensity has not yet been calculated and remove polygons that are not in Peel
polygons_save = polygons_aggregated[polygons_aggregated["norm_var"].notna()]
# check length
print(len(np.unique(polygons_save.id)))
# save
#polygons_save.to_csv(f'{site}_{name}.csv', index=True)

99


***

Read volume

In [13]:
data_ = polygons_save #pd.read_csv(f"{site}_{name}.csv")
print(data_.year_start.unique(), data_.year_end.unique())
print('total volume: ', np.round(data_.volume.sum()))

99
[2010] [2021 2016]
1465625.0


In [33]:
end_ = data_.year_end.unique()
data1 = data_[(data_.year_end == end_[0])] # | (data_.year_end == end_[2]) ]

[2010] [2021 2016]


In [35]:
print('volume with end',end_[0], 'is:', np.round(data1.volume.sum()))
print('mean area', np.round(data1.area.mean()))

volume 2215977.0
mean area 5108.0
